In [2]:
import numpy as np
import pandas as pd
from sklearn import svm, datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [3]:
columns = ["sex","length","diam","height","whole","shucked","viscera","shell","age"]
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",names=columns)
df.head()

,sex,length,diam,height,whole,shucked,viscera,shell,age
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [6]:
y = df.age
X=df.drop(columns=['age'])
num_cols = x.select_dtypes(include=np.number).columns
cat_cols = x.select_dtypes(include=['object']).columns
num_cols, cat_cols

(Index(['length', 'diam', 'height', 'whole', 'shucked', 'viscera', 'shell'], dtype='object'),
 Index(['sex'], dtype='object'))

In [7]:
#create some missing values
for i in range(1000):
    x.loc[np.random.choice(x.index),np.random.choice(x.columns)] = np.nan
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.25)

In [14]:
cat_vals = Pipeline([("imputer",SimpleImputer(strategy='most_frequent')), ("ohe",OneHotEncoder(sparse_output=False, drop='first'))])
num_vals = Pipeline([("imputer",SimpleImputer(strategy='mean')), ("scale",StandardScaler())])
cat_vals, num_vals

(Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                 ('ohe', OneHotEncoder(drop='first', sparse_output=False))]),
 Pipeline(steps=[('imputer', SimpleImputer()), ('scale', StandardScaler())]))

In [15]:
preprocess = ColumnTransformer(transformers=[
    ("cat_process", cat_vals, cat_cols),
    ("num_process", num_vals, num_cols)
])

In [16]:
#1 create a pipeline with the preprocess and a linear regression model
pipeline = Pipeline([("preprocess", preprocess), ("regress", LinearRegression())])

In [17]:
#2. Fit the pipeline on the training data
pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat_process',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  Index(['sex'], dtype='object')),
                                                 ('num_process',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['length', 'diam', 'height', 'whole', 'shucked', 'viscera', 'shell'], dtype='object'))])),
                ('regress', LinearRegression())])

In [18]:
#3. predict the pipeline on the test data
pipeline.predict(x_test)

array([13.22112125,  9.11163817, 10.32787122, ..., 10.22454563,
       12.56346187, 12.36977818])